In [21]:
import pandas as pd

df = pd.read_excel('excel/coupang2024.xlsx')
df2023=pd.read_excel('excel/coupang2023.xlsx')

df,df2023

(                     주문일                                  상품명      가격  \
 0    2025-01-15 02:19:03                           자가비 케첩맛 3p   7,180   
 1    2025-01-15 02:19:03                       농심 육개장 사발면 86g   4,980   
 2    2025-01-15 02:19:03                    오뚜기 컵누들 마라탕 44.7g   6,350   
 3    2025-01-15 02:19:03         라온 2in1 맥세이프 호환 듀얼 무선충전기 15W  27,900   
 4    2025-01-13 05:06:48  솔향가득 잎편백수 천연 피톤치드 원액 스프레이 + 스프레이 공병  16,500   
 ..                   ...                                  ...     ...   
 413  2024-01-02 05:31:02                         나랑드 사이다 그린애플  13,200   
 414  2024-01-02 05:31:02            코멧 순백 3겹 라벤더 바닐라 롤화장지 30m  13,490   
 415  2024-01-02 05:31:02          코멧 프리미엄 부드러운 천연펄프 미용티슈 280매  10,890   
 416  2024-01-02 05:31:02                      슈가로로 스파클링 사과 음료  22,900   
 417  2024-01-02 05:31:02                카누 다크로스트 아메리카노 원두커피믹스  38,360   
 
           카테고리  
 0      음식 및 음료  
 1      음식 및 음료  
 2      음식 및 음료  
 3    장비 및 생활용품  
 4    장비 및 생활용품  
 .

In [22]:
import numpy as np

# 빈 문자열을 NaN으로 변환
df2023.replace("", np.nan, inplace=True)

# 비어있는 행
df2023.isnull().sum()

주문일     0
상품명     0
가격      0
카테고리    0
dtype: int64

In [23]:
# NaN이 포함된 행 삭제
df2023.dropna(how='all', inplace=True)

print(df2023.head())  # 결과 확인
print(df2023.isnull().sum())

                   주문일                          상품명      가격     카테고리
0  2023-12-21 08:09:34        켈로그 프로틴 그래놀라 쉐이크 미숫가루  19,900  음식 및 음료
1  2023-12-21 08:09:34                     쿠크다스 케이크  13,020  음식 및 음료
2  2023-12-21 08:09:34  미주라 통밀도넛츠 2p + 초코칩 통밀도넛츠 2p  17,500  음식 및 음료
3  2023-12-21 08:09:34               곰곰 간편한 미숫가루 스틱  16,300  음식 및 음료
4  2023-12-21 08:09:34               농심 짜파게티 범벅 70g   5,040  음식 및 음료
주문일     0
상품명     0
가격      0
카테고리    0
dtype: int64


In [20]:
#저장 
# df2023.to_excel('coupang2023.xlsx', index=False)

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


# 라벨 인코딩
df['카테고리'] = df['카테고리'].astype('category')
label_mapping = {idx: label for idx, label in enumerate(df['카테고리'].cat.categories)}
reverse_label_mapping = {label: idx for idx, label in label_mapping.items()}
df['카테고리'] = df['카테고리'].cat.codes

print("Label Mapping:", label_mapping)
print("Reverse Label Mapping:", reverse_label_mapping)

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df['상품명'], df['카테고리'], test_size=0.2, random_state=42)

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 로지스틱 회귀 모델 학습
model = LogisticRegression(max_iter=1000, random_state=42,class_weight='balanced')
model.fit(X_train_tfidf, y_train)

# 예측
y_pred = model.predict(X_test_tfidf)

# 성능 평가
print("\n모델 성능 평가:")
print(classification_report(y_test, y_pred, target_names=list(label_mapping.values()), zero_division=0))

# 새로운 데이터 예측
df_tfidf = vectorizer.transform(df2023['상품명'])
df2023['카테고리'] = model.predict(df_tfidf)
df2023['카테고리'] = df2023['카테고리'].map(label_mapping)

# 결과 확인
print("\n예측된 결과:")
print(df)


Label Mapping: {0: '음식 및 음료', 1: '장비 및 생활용품'}
Reverse Label Mapping: {'음식 및 음료': 0, '장비 및 생활용품': 1}

모델 성능 평가:
              precision    recall  f1-score   support

     음식 및 음료       0.88      1.00      0.94        59
   장비 및 생활용품       1.00      0.68      0.81        25

    accuracy                           0.90        84
   macro avg       0.94      0.84      0.87        84
weighted avg       0.92      0.90      0.90        84


예측된 결과:
                     주문일                                  상품명      가격  카테고리
0    2025-01-15 02:19:03                           자가비 케첩맛 3p   7,180     0
1    2025-01-15 02:19:03                       농심 육개장 사발면 86g   4,980     0
2    2025-01-15 02:19:03                    오뚜기 컵누들 마라탕 44.7g   6,350     0
3    2025-01-15 02:19:03         라온 2in1 맥세이프 호환 듀얼 무선충전기 15W  27,900     1
4    2025-01-13 05:06:48  솔향가득 잎편백수 천연 피톤치드 원액 스프레이 + 스프레이 공병  16,500     1
..                   ...                                  ...     ...   ...
413  2024-01-02 05:31

In [12]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 로지스틱 회귀 모델 학습 및 GridSearchCV
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['liblinear'],
    'class_weight': ['balanced']
}

grid = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_tfidf, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid.best_params_)

# 최적의 모델로 예측
y_pred = grid.best_estimator_.predict(X_test_tfidf)

# 성능 평가
print("\n모델 성능 평가:")
print(classification_report(y_test, y_pred, target_names=list(label_mapping.values()), zero_division=0))

Best Parameters: {'C': 1, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}

모델 성능 평가:
              precision    recall  f1-score   support

     음식 및 음료       0.88      1.00      0.94        59
   장비 및 생활용품       1.00      0.68      0.81        25

    accuracy                           0.90        84
   macro avg       0.94      0.84      0.87        84
weighted avg       0.92      0.90      0.90        84



In [13]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 로지스틱 회귀 모델 학습 및 GridSearchCV
param_grid = {
    'C': [0.3,0.4,0.5,0.6,0.7],
    'penalty': ['l2'],
    'solver': ['liblinear'],
    'class_weight': ['balanced']
}

grid = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), param_grid, cv=5, scoring='accuracy')
grid.fit(X_train_tfidf, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Parameters:", grid.best_params_)

# 최적의 모델로 예측
y_pred = grid.best_estimator_.predict(X_test_tfidf)

# 성능 평가
print("\n모델 성능 평가:")
print(classification_report(y_test, y_pred, target_names=list(label_mapping.values()), zero_division=0))

Best Parameters: {'C': 0.4, 'class_weight': 'balanced', 'penalty': 'l2', 'solver': 'liblinear'}

모델 성능 평가:
              precision    recall  f1-score   support

     음식 및 음료       0.88      1.00      0.94        59
   장비 및 생활용품       1.00      0.68      0.81        25

    accuracy                           0.90        84
   macro avg       0.94      0.84      0.87        84
weighted avg       0.92      0.90      0.90        84



In [14]:
# 새로운 데이터 예측 2023데이터터
df_tfidf = vectorizer.transform(df2023['상품명'])
df2023['카테고리'] = grid.best_estimator_.predict(df_tfidf)
df2023['카테고리'] = df2023['카테고리'].map(label_mapping)

# 결과 확인
print("\n예측된 결과:")
print(df2023)


예측된 결과:
                     주문일                          상품명      가격       카테고리
0    2023-12-21 08:09:34        켈로그 프로틴 그래놀라 쉐이크 미숫가루  19,900    음식 및 음료
1    2023-12-21 08:09:34                     쿠크다스 케이크  13,020    음식 및 음료
2    2023-12-21 08:09:34  미주라 통밀도넛츠 2p + 초코칩 통밀도넛츠 2p  17,500    음식 및 음료
3    2023-12-21 08:09:34               곰곰 간편한 미숫가루 스틱  16,300    음식 및 음료
4    2023-12-21 08:09:34               농심 짜파게티 범벅 70g   5,040    음식 및 음료
..                   ...                          ...     ...        ...
409  2023-01-09 01:10:56        카누 다크로스트 아메리카노 원두커피믹스  16,340    음식 및 음료
410  2023-01-08 03:56:31                고르고 바른 마스크 대형   7,900  장비 및 생활용품
411  2023-01-05 02:42:00                   공차 허니자몽블랙티   7,380    음식 및 음료
412  2023-01-05 02:42:00           ipTIME USB허브 UC304  14,240  장비 및 생활용품
413  2023-01-05 02:42:00        알티스트 핫초코 한잔에 당분이 단 1g   8,900    음식 및 음료

[414 rows x 4 columns]


In [15]:
# 엑셀 파일로 저장 
df2023.to_excel('coupang2023.xlsx', index=False)